# New-RscMssqlSnapshot
DBAs will often take a backup before they make a change to the database. In general, you need 2 things to take a snapshot of a database, the database ID and the SLA ID. An SLA is required for all on demand snapshots to manage the retention of the snapshot. 

# Workflow
To start an On-Demand Snapsot of a database, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscHost** to get the id of the host. This will be used in the next cmdlet when looking up a SQL Instance
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Get-RscMssqlDatabase** to get the database ID based on the database name and the SQL Server Instance. 
1. Use **Get-RscSLADomain** to get the ID SLA Domain that will manage the retention of the snapshot.
1. Use **New-RscMssqlSnapshot** to initiate an On Demand Snapshot of a database. 
1. Use **Get-RscRequest** to track the request and know when it completes. ***DOES NOT EXIST YET***
1. Use **Disconnect-RSC** to disconnect from RSC

## Connect to RSC

In [1]:
Import-Module RubrikSecurityCloud
Connect-RSC


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20240219-20     v20240304-21



## Get the Cluster ID

In [2]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Host ID

In [3]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows

## Get the Instance Information

In [4]:
$RscMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Get the Database Information

In [5]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName -RscMssqlInstance $RscMssqlInstance
$RscMssqlDatabase


AuthorizedOperations            : 
ObjectType                      : MSSQL
SlaAssignment                   : DIRECT
ConfiguredSlaDomain             : 
EffectiveRetentionSlaDomain     : 
EffectiveSlaDomain              : RubrikSecurityCloud.Types.GlobalSlaReply
PendingSla                      : 
ReplicatedObjects               : 
CdmId                           : 42527305-5e85-4777-8689-955237d3d951
CdmLink                         : 
CdmOnDemandSnapshotCount        : 
CopyOnly                        : False
DagId                           : 9024703f-4510-5209-b3bb-52b7115ad5b9
HasLogConfigFromSla             : True
HasPermissions                  : True
HostLogRetention                : -1
Id                              : a3ca658f-3a80-5a3f-8108-a2156e5fd6a3
IsInAvailabilityGroup           : False
IsLogShippingSecondary          : False
IsMount                         : False
IsOnline                        : True
IsRelic                         : False
LogBackupFrequencyInSeconds    

## Get SLA Domain Information

In [6]:
$SLADomain = "MSSQL-1d-30d-NoArchive-Repl-Houghton"
$RscSlaDomain = Get-RscSla -Name $SLADomain
$RscSlaDomain


Name                                 Base Frequency Object Count Archival Location Replication
                                                                                   Target
----                                 -------------- ------------ ----------------- ----------------
MSSQL-1d-30d-NoArchive-Repl-Houghton 1 DAYS         0                                              



Error: Command failed: SubmitCode: $SLADomain = "MSSQL-1d-30d-NoArchive-Repl-Houghton ...

## Initiate On Demand Snapshot

In [7]:
New-RscMssqlSnapshot -RscMssqlDatabase $RscMssqlDatabase -RscSlaDomain $RscSlaDomain


EndTime   : 
Id        : MSSQL_DB_BACKUP_138e0fbc-c0f8-4df2-97b4-3338258e49c1_82340416-343c-4e2b-b323-abda7c668a
            84:::0
NodeId    : 
Progress  : 
StartTime : 
Status    : QUEUED
Error     : 
Links     : 



## Track the status of the backup

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [8]:
Disconnect-Rsc

The Rubrik Security Cloud session has been terminated.
